# YamboConvergence: automated BSE convergence

The highest level workflow is represented by the ``YamboConvergence`` workchain, 
which implements the full automation of the convergence algorithm described in [Bonacci, M., Qiao, J., Spallanzani, N. et al. Towards high-throughput many-body perturbation theory: efficient algorithms and automated workflows. npj Comput Mater 9, 74 (2023)](https://doi.org/10.1038/s41524-023-01027-2). 
Simulations are organized on the fly, without any external user intervention. 
The purpose of this new proposed convergence algorithm is to obtain an accurate converged 
result doing the least possible number of calculations. This is possible if a reliable description of the convergence space is achieved, resulting also in a 
precise guess for the converged point, i.e. the converged parameters. The description of the space is performed by fitting some calculations that the workchain runs. 
A simple functional form of the space is assumed:

$f(\textbf{x}) = \prod_i^N \left( \frac{A_i}{x_i^{\alpha_i}} + b_i \right)$

where N is the number of parameters which we are converging simultaneously.
In this way it is straightforward to compute first and second partial derivatives, and impose constraints on them to find the converged region of the space. 
The algorithm can be used to easily perform convergence tests with respect to the ``k-point mesh``.

Each simulation is performed by calling the `YamboWorkflow` workchain, ensuring the automation at all levels.

In [69]:
from aiida import orm, load_profile
load_profile()

Profile<uuid='b35700dae723411ea16ebc82d58f16bc' name='mb'>

In [70]:
from aiida.plugins import WorkflowFactory

In [71]:
YamboConvergence = WorkflowFactory('yambo.yambo.yamboconvergence')

## minimal inputs needed for protocols

In [72]:
options = {
    'pwcode_id': 'pw-7.1@hydralogin', 
    'pseudo_family':"PseudoDojo/0.4/PBE/SR/standard/upf",
    'yamboprecode_id':'p2y-5.1@hydralogin',
    'yambocode_id':'yambo-5.1@hydralogin',
    'protocol':'fast',
    #'parent_id':274, #not necessary to set; take your previously nscf id (pk) to skip the DFT part.
    'structure_id':161,
}

In [74]:
from aiida_quantumespresso.common.types import ElectronicType

In [75]:
builder = YamboConvergence.get_builder_from_protocol(
            pw_code = options['pwcode_id'],
            preprocessing_code = options['yamboprecode_id'],
            code = options['yambocode_id'],
            protocol=options['protocol'],
            protocol_qe=options['protocol'],
            structure=orm.load_node(options['structure_id']),
            overrides={},
            #parent_folder=load_node(options['parent_id']).outputs.remote_folder,
            electronic_type=ElectronicType.INSULATOR, #default is METAL: smearing is used
            calc_type='bse', #or 'bse'; default is 'gw'
)

Summary of the main inputs:
BndsRnXs = 200
NGsBlkXs = 6 Ry
BSENGBlk = 6 Ry
FFTGvecs = 18 Ry


kpoint mesh for nscf: [6, 6, 2]


/scratch/mbonacci/mambaforge/envs/minitutorial/lib/python3.10/site-packages/aiida_quantumespresso/calculations/pw.py:211: UserWarning: `parent_folder` not provided for `nscf` calculation. For work chains wrapping this calculation, you can disable this warning by excluding the `parent_folder` when exposing the inputs of the `PwCalculation`.
  warnings.warn(


In [76]:
#You can also try different protocols:
    
YamboConvergence.get_available_protocols()

{'fast': {'description': 'Fast protocol for a GW convergence: grid -> poor; thresholds -> poor'},
 'moderate': {'description': 'Moderate protocol for a GW convergence: grid -> enough good for standard materials; thresholds -> moderate (5 percent)'},
 'precise': {'description': 'precise protocol for a GW convergence: grid -> same as moderate; thresholds -> precise (1 percent)'},
 'molecule': {'description': 'Moderate protocol for a GW convergence in molecules'}}

Now, if you inspect the prepopulated inputs, you can see the default values respecting the imposed protocol:

In [77]:
builder.ywfl.nscf.pw.parameters.get_dict()

{'CONTROL': {'calculation': 'nscf',
  'forc_conv_thr': 0.001,
  'tprnfor': True,
  'tstress': True,
  'etot_conv_thr': 0.0004},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'ecutwfc': 60.0,
  'ecutrho': 480.0,
  'force_symmorphic': True,
  'nbnd': 200},
 'ELECTRONS': {'electron_maxstep': 80,
  'mixing_beta': 0.4,
  'conv_thr': 1.6e-09}}

In [78]:
builder.ywfl.yres.yambo.parameters.get_dict()

{'arguments': ['em1s', 'bse', 'bss', 'optics', 'rim_cut', 'dipoles'],
 'variables': {'Chimod': 'hartree',
  'DysSolver': 'n',
  'GTermKind': 'BG',
  'X_and_IO_nCPU_LinAlg_INV': [1, ''],
  'BSEmod': 'resonant',
  'BSKmod': 'SEX',
  'BSSmod': 'd',
  'Lkind': 'full',
  'BSEQptR': [[1, 1], ''],
  'FFTGvecs': [18, 'Ry'],
  'GbndRnge': [[1, 200], ''],
  'BndsRnXs': [[1, 200], ''],
  'NGsBlkXs': [6, 'Ry'],
  'BSENGBlk': [6, 'Ry']}}

### Computational resources


In [79]:
builder.ywfl.scf.pw.metadata.options = {
    'max_wallclock_seconds': 2*60*60, # in seconds
    'resources': {
            "num_machines": 1, # nodes
            "num_mpiprocs_per_machine": 16, # MPI per nodes
            "num_cores_per_mpiproc": 1, # OPENMP
        },
    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed
    #'account':'project_name',
    'queue_name':'s3par',
    #'qos':'',
}

builder.ywfl.nscf.pw.metadata.options = builder.ywfl.scf.pw.metadata.options
builder.ywfl.yres.yambo.metadata.options = builder.ywfl.scf.pw.metadata.options

### Overrides

It is possible to modify the default inputs also during the builder creation phase, so not a posteriori. This can be done by using overrides:

In [124]:
overrides_scf = {
        'pseudo_family': "PseudoDojo/0.4/PBE/SR/standard/upf", 
        'pw':{
            
        'metadata':{
                    'options':{
                    'max_wallclock_seconds': 60*60, # in seconds
                    'resources': {
                            "num_machines": 1, # nodes
                            "num_mpiprocs_per_machine": 16, # MPI per nodes
                            "num_cores_per_mpiproc": 1, # OPENMP
                        },
                    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed
                    #'account':'project_name',
                    'queue_name':'s3par',
                    #'qos':'',
                                    },
        },
        },
    }

overrides_nscf = {
        'pseudo_family': "PseudoDojo/0.4/PBE/SR/standard/upf", 
        'pw': {
            'parameters':{
                'CONTROL':{}, #not needed if you don't override something
                'SYSTEM':{},
                'ELECTRONS':{'diagonalization':'cg'},
            },
             'metadata':{
                    'options':{
                    'max_wallclock_seconds': 60*60, # in seconds
                    'resources': {
                            "num_machines": 1, # nodes
                            "num_mpiprocs_per_machine": 16, # MPI per nodes
                            "num_cores_per_mpiproc": 1, # OPENMP
                        },
                    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed
                    #'account':'project_name',
                    'queue_name':'s3par',
                    #'qos':'',
                                    },
        },
    },
}

overrides_yambo = {
        "yambo": {
            "parameters": {
                "arguments": [
                    "rim_cut",
                ],
                "variables": {
                    "FFTGvecs": [20, "Ry"],
                    'GbndRnge': [[1, 50], ''],
                    'BndsRnXs': [[1, 50], ''],
                    'NGsBlkXs': [2, 'Ry'],
                    'BSENGBlk': [2, 'Ry'],
                    'KfnQP_E':[[1.5,1,1],''],           # <== Scissor and stretching correction.
                    'BS_CPU':str(int(16/2))+' 2 1',     # <== PARALLELISM INFO
                    'BS_ROLEs':'k eh t',                # <== PARALLELISM INFO
                },
            },
        'metadata':{
                    'options':{
                    'max_wallclock_seconds': 60*60, # in seconds
                    'resources': {
                            "num_machines": 1, # nodes
                            "num_mpiprocs_per_machine": 16, # MPI per nodes
                            "num_cores_per_mpiproc": 1, # OPENMP
                        },
                    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed, i.e. in PBS/Torque 
                    #'account':'project_name',
                    'queue_name':'s3par',
                    #'qos':'',
                                    },
                    },
        },
    
}

#Be careful with the mesh choice!!! 
overrides_meta = {
        'FFTGvecs': {
            'start_ratio': 0.25,
            'stop_ratio': 0.7,
            'delta_ratio': 0.1,
            'max_ratio': 1,
        },
        'bands': {
            'start': 50,
            'stop': 400,
            'delta': 50,
            'max': 600,
            'ratio':[10,25,50],
        },
        'G_vectors': {
            'start': 2,
            'stop': 8,
            'delta': 1,
            'max': 10,
        },
        'kpoint_density': {
            'start': 0.8,
            'stop': 0.2,
            'delta': 3,
            'max': 0.1,
        } ,
        'conv_thr_k': 5,   # <== 5% for convergence wrt super-converged estimation.
        'conv_thr_bG': 10,
        'conv_thr_FFT': 10,
        'conv_thr_units': '%', # 'eV'

        
    }

        
overrides_wfl_settings = {
        
        'what':['lowest_exciton'],          # <== converging the lowest exciton.
        'bands_nscf_update':'full-step',
        'skip_pre':False,   
        'type': 'heavy', #or cheap; heavy uses converged value for parameters that we are not converging in a given iteration.
        
    }

overrides = {
        'meta_parameters':overrides_meta,
        'ywfl':{'scf':overrides_scf,'nscf':overrides_nscf,'yres':overrides_yambo},
        'workflow_settings':overrides_wfl_settings,
    }


In [125]:
builder = YamboConvergence.get_builder_from_protocol(
            pw_code = options['pwcode_id'],
            preprocessing_code = options['yamboprecode_id'],
            code = options['yambocode_id'],
            protocol=options['protocol'],
            protocol_qe=options['protocol'],
            structure= orm.load_node(options['structure_id']),
            overrides=overrides,
            #parent_folder=load_node(options['parent_id']).outputs.remote_folder,
            electronic_type=ElectronicType.INSULATOR, #default is METAL: smearing is used
            calc_type='bse', #or 'bse'; default is 'gw'
)

family = orm.load_group("PseudoDojo/0.4/PBE/SR/standard/upf")
#builder.<sublevels_up_to .pw>.pseudos = family.get_pseudos(structure=structure) 
builder.ywfl.scf.pw.pseudos = family.get_pseudos(structure=orm.load_node(161)) 
builder.ywfl.nscf.pw.pseudos = family.get_pseudos(structure=orm.load_node(161)) 

Summary of the main inputs:
BndsRnXs = 50
NGsBlkXs = 2 Ry
BSENGBlk = 2 Ry
FFTGvecs = 20 Ry


kpoint mesh for nscf: [6, 6, 2]


/scratch/mbonacci/mambaforge/envs/minitutorial/lib/python3.10/site-packages/aiida_quantumespresso/calculations/pw.py:211: UserWarning:

`parent_folder` not provided for `nscf` calculation. For work chains wrapping this calculation, you can disable this warning by excluding the `parent_folder` when exposing the inputs of the `PwCalculation`.



In [126]:
try:
    g = orm.load_group('tutorial/hBN/convergence/BSE')
except:
    g = orm.Group('tutorial/hBN/convergence/BSE')
    g.store()

In [127]:
builder.group_label = orm.Str('tutorial/hBN/convergence/BSE') # verdi group create tutorial/hBN/convergence; all calculationsc are added to the group

In [128]:
builder.ywfl.yres.yambo.parameters = orm.Dict(
    dict={'arguments':['em1s','bse','bss','optics', 'dipoles',],
                'variables':{
                'BSEmod': 'resonant',
                'BSKmod': 'SEX',
                'BSSmod': 'd',
                'Lkind': 'full',
                'NGsBlkXs': [2, 'Ry'],
                'BSENGBlk': [2, 'Ry'],
                'Chimod': 'hartree',
                'DysSolver': 'n',
                'BEnSteps': [10,''],
                'BSEQptR': [[1,1],''],
                'BSEBands': [[8,9],''],
                'BEnRange': [[0.0, 10.0],'eV'],
                'BDmRange': [[0.1, 0.1],'eV'],
                'BLongDir': [[1.0, 1.0, 1.0],''],
                'LongDrXp': [[1.0, 1.0, 1.0],''],
                'LongDrXd': [[1.0, 1.0, 1.0],''],
                'LongDrXs': [[1.0, 1.0, 1.0],''],
                'BndsRnXs': [[1,50], ''],
                'KfnQP_E':[[1.5,1,1],''],
                'BS_CPU':str(int(16/2))+' 2 1',
                'BS_ROLEs':'k eh t',
                },}
)

In [129]:
builder.parameters_space.get_list()

[{'var': ['FFTGvecs'],
  'start': 21,
  'stop': 58,
  'delta': 8,
  'max': 84,
  'steps': 4,
  'max_iterations': 4,
  'conv_thr': 10,
  'conv_thr_units': '%',
  'convergence_algorithm': 'new_algorithm_1D'},
 {'var': ['kpoint_mesh'],
  'start': [4, 4, 2],
  'stop': [16, 16, 6],
  'delta': [3, 3, 3],
  'max': [30, 30, 10],
  'steps': 4,
  'max_iterations': 4,
  'conv_thr': 5,
  'conv_thr_units': '%',
  'convergence_algorithm': 'new_algorithm_1D'},
 {'var': ['BndsRnXp', 'GbndRnge', 'NGsBlkXp'],
  'start': [80, 80, 2],
  'stop': [400, 400, 8],
  'delta': [50, 50, 1],
  'max': [600, 600, 10],
  'steps': 6,
  'max_iterations': 8,
  'conv_thr': 10,
  'conv_thr_units': '%',
  'convergence_algorithm': 'new_algorithm_2D'}]

In [130]:
builder.parameters_space = orm.List(builder.parameters_space.get_list()[-2:-1])

In [131]:
builder.parameters_space.get_list()


[{'var': ['kpoint_mesh'],
  'start': [4, 4, 2],
  'stop': [16, 16, 6],
  'delta': [3, 3, 3],
  'max': [30, 30, 10],
  'steps': 4,
  'max_iterations': 4,
  'conv_thr': 5,
  'conv_thr_units': '%',
  'convergence_algorithm': 'new_algorithm_1D'}]

### Overrides

It is possible to modify the default inputs also during the builder creation phase, so not a posteriori. This can be done by using overrides:

In [121]:
from aiida.engine import submit

In [122]:
run = None

In [123]:
if run:
    print('run is already running -> {}'.format(run.pk))
    print('sure that you want to run again?, if so, copy the else instruction in the cell below and run!')
else:
    run = submit(builder)

print(run)

/scratch/mbonacci/mambaforge/envs/minitutorial/lib/python3.10/site-packages/aiida_quantumespresso/calculations/pw.py:211: UserWarning:

`parent_folder` not provided for `nscf` calculation. For work chains wrapping this calculation, you can disable this warning by excluding the `parent_folder` when exposing the inputs of the `PwCalculation`.



uuid: 81ddbad2-2e88-4e27-bee8-a6c6fe4dbd09 (pk: 6595) (aiida.workflows:yambo.yambo.yamboconvergence)


# Output analysis.

suppose that your calculation completed successfully, then you can access the outputs via the output method of the run instance: 

In [133]:
run.is_finished_ok

True

The converged parameters can be obtained via the "infos" output Dict:

In [134]:
run.outputs.infos.get_dict()

{'E_ref': 5.6156411578646,
 'kpoint_mesh': [7, 7, 4],
 'extrapolation': 5.6175120483652,
 'lowest_exciton': 5.5455183982849}

The full convergence history can be visualized in a table form using pandas:

In [135]:
import pandas as pd

In [136]:
history = run.outputs.history.get_dict()

In [137]:
history_table = pd.DataFrame(history)

In [138]:
history_table

,uuid,failed,useful,global_step,kpoint_mesh,lowest_exciton,parameters_studied
0,1b01b60f-bf26-4c4a-be35-d145547e635c,False,False,1,"[4, 4, 2]",5.359427,kpoint_mesh
1,37fcb1a7-b998-44d6-8c57-89facc0f4f90,False,True,2,"[7, 7, 2]",5.545518,kpoint_mesh
2,a0414f3e-09c3-47e8-b573-830d2f776c71,False,False,3,"[13, 13, 5]",5.598341,kpoint_mesh
3,3aeb473e-6747-4397-8802-c4cdbcfba2a3,False,False,4,"[16, 16, 6]",5.623181,kpoint_mesh
4,e1fc6ebf-06f6-40f4-ac99-38d6375e473a,False,False,5,"[7, 7, 4]",5.471172,kpoint_mesh


The last calculations can be obtained using:

In [139]:
history_table[history_table['useful']==True]

,uuid,failed,useful,global_step,kpoint_mesh,lowest_exciton,parameters_studied
1,37fcb1a7-b998-44d6-8c57-89facc0f4f90,False,True,2,"[7, 7, 2]",5.545518,kpoint_mesh


Result on the convergence path can be plotted using several plotting libraries, for examples here we are gonna use [plotly]((https://plotly.com/python/3d-line-plots/)) to observe the convergence between bands and plane wave cutoff for the screening matrix:

In [143]:
import plotly.express as px

k_history = history_table[history_table['parameters_studied']=="kpoint_mesh"]

df = k_history
fig = px.scatter(df, y="lowest_exciton", text=df["kpoint_mesh"])
fig.show()

## Exercise 1: try to compute BSE convergence but using QP instead of scissor and stretching.

Solution provided in the `Solution_2_YamboConvergence_BSE_QP.ipynb` notebook. 
Please look at the `6_1_YamboWorkflow_BSE_QP.ipynb` to understand how to set the preliminary QP calculation.

## Exercise 2: computing the convergence not with respect to the k-mesh, but with respect to the k-point density.

```python
builder.parameters_space = orm.List(list=[
 {'var': ['kpoint_density'],
  'start': 1/0.05,
  'stop': 1/0.025,
  'delta': 1,      #1/0.01
  'max': 1/0.02,
  'steps': 4,
  'max_iterations': 4,
  'conv_thr': 3,
  'conv_thr_units': '%',
  'convergence_algorithm': 'new_algorithm_1D'},])
```